<a href="https://colab.research.google.com/github/mostafa-sadeghi/amirali_eslami/blob/main/another_way_vgg16oncifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def preprocess_data(X, Y):
    """
    function that pre-processes the CIFAR10 dataset 
    """
    X = K.applications.vgg16.preprocess_input(X)
    Y = K.utils.to_categorical(Y)
    return X, Y

In [6]:
# load the Cifar10 dataset, 50,000 training images
# and 10,000 test images (here used as validation data)
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
# preprocess the data using the application's preprocess_input method and convert the labels to one-hot encodings
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)

In [8]:
initializer = K.initializers.he_normal()
input_tensor = K.Input(shape=(32, 32, 3))
# resize images to the image size upon which the network

resized_images = K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_tensor)
model = K.applications.VGG16(include_top=False,
                                   weights='imagenet',
                                   input_tensor=resized_images,
                                   input_shape=(224, 224, 3),
                                   pooling='max',
                                   classes=1000)
# make the weights and biases of the base model non-trainable

for layer in model.layers:
    layer.trainable = False
output = model.layers[-1].output
# reshape the output feature map of the base model before passing the data on to the Dense layers of the classifier head
flatten = K.layers.Flatten()
output = flatten(output)
layer_256 = K.layers.Dense(units=256,
                           activation='elu',
                           kernel_initializer=initializer,
                           kernel_regularizer=K.regularizers.l2())
output = layer_256(output)
dropout = K.layers.Dropout(0.5)
output = dropout(output)
softmax = K.layers.Dense(units=10,
                         activation='softmax',
                         kernel_initializer=initializer,
                         kernel_regularizer=K.regularizers.l2())
output = softmax(output)
model = K.models.Model(inputs=input_tensor, outputs=output)

58889256/58889256 [==============================] - 2s 0us/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [9]:
model.compile(
         optimizer=K.optimizers.Adam(learning_rate=1e-4),
         loss='categorical_crossentropy',
         metrics=['accuracy'])

In [10]:
# stop training when val_accuracy has stopped improving
early_stop = K.callbacks.EarlyStopping(monitor='val_accuracy',
                                       patience=3,
                                       verbose=1,
                                       mode='max')

In [ ]:
history = model.fit(x_train,y_train,
                    validation_data=(x_validate, y_validate),
                    epochs=100,
                    callbacks=[early_stop],
                    verbose=1)


Epoch 1/100
  14/1250 [..............................] - ETA: 5:17:48 - loss: 54.7624 - accuracy: 0.1116

In [ ]:
evaluation = model.evaluate(x_test,y_test)
print(f"Test accuracy: {evaluation[1]*100:.2f}%")

In [ ]:
predicted_classes = model.predict(x_test)
len(predicted_classes)

In [ ]:
print(predicted_classes[0])
y_test[0]

In [ ]:
np.argmax(predicted_classes[0])

In [ ]:
y_test[0]

In [ ]:
fig, axes = plt.subplots(5,5,figsize=(12,12))
axes = axes.ravel()
print(axes.shape)
x_test[0].shape
for i in np.arange(0,25):
  axes[i].imshow(x_test[i].reshape(28,28))
  axes[i].set_title(f"Prediction Class = {np.argmax(predicted_classes[i]):.1f}\nTrue class = {y_test[i]}")


  axes[i].axis('off')

plt.subplots_adjust(wspace=0.5)

In [ ]:
from vis.utils import utils

In [ ]:
# swap softmax layer with linear layer 
layer_idx = utils.find_layer_idx(model, 'predictions')
model.layers[-1].activation = tf.keras.activations.linear
model = utils.apply_modifications(model)

In [ ]:
!pip install tf-keras-vis

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

In [ ]:
gradCam = Gradcam(model, clone =True)

In [ ]:
from matplotlib.pyplot import plt

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore

score = CategoricalScore([3, 8, 8 , 0])

In [ ]:
input_classes = ['cat', 'ship', 'ship', 'airplain']

In [ ]:
images = [x_test[0], x_test[1], x_test[2], x_test[3]]

In [ ]:
y_test[3]

In [ ]:
plt.imshow(x_test[100])

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
input_images = preprocess_input(images)

In [ ]:
cam = gradCam(score, input_images, penultimate_layer=-1)

In [ ]:
f, ax = plt.subplots(nrows = 1, ncols = 4, figsize=(12,4))
for i, img_class in enumerate(input_classes):
    heatmap = np.uint8(cm.jet(cam[i])[..., :4] * 255)
    ax[i].set_title(img_class, fontsize=16)
    ax[i].imshow(x_test[i])
    ax[i].imshow(heatmap, cmap='jet', alpha=0.2) 
    ax[i].axis('off')
plt.tight_layout()
plt.show()